In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('../results/cleaned_data.csv')
data.head()

,timestamp,user_id,day,quiz_id,image_index,illusion_filename,user_answer,response_time,speed_group,display_time,correct_answer,line_length1,line_length2,actual_difference,arrow_length,angle,arrow_color,is_control,is_correct
0,2024-11-05 04:53:53,Bchris17,1,0,0,muller_lyer_day1_illusion5.svg,Left,3.534,Group 1 - Fast,0.5,Same length,210,210,0,20,36,black,False,False
1,2024-11-05 04:53:57,Bchris17,1,0,1,muller_lyer_day1_illusion10.svg,Same length,2.622,Group 1 - Fast,0.5,Same length,185,185,0,20,36,black,False,True
2,2024-11-05 04:54:02,Bchris17,1,0,2,muller_lyer_day1_illusion32.svg,Left,3.985,Group 1 - Fast,0.5,Same length,215,215,0,20,36,red,False,False
3,2024-11-05 04:54:06,Bchris17,1,0,3,muller_lyer_day1_illusion19.svg,Right,2.841,Group 1 - Fast,0.5,Right,200,220,-20,20,36,black,False,True
4,2024-11-05 04:54:10,Bchris17,1,0,4,muller_lyer_day1_illusion8.svg,Left,2.837,Group 1 - Fast,0.5,Same length,215,215,0,20,36,black,False,False
